In [1]:
!pip install tfrecord wandb torch_dct

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 15.0 MB/s eta 0:00:00


In [1]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '/notebooks'

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX'

import os
import sys
import torch
import torch.nn as nn

from os.path import join as ospj
sys.path.append(ospj(PROJECT_ROOT, 'src'))
%load_ext autoreload
%autoreload 2

In [2]:
import gc

trainer.model.cpu()
del trainer.model
del trainer

gc.collect()
torch.cuda.empty_cache()
del train_loader
del val_loader
gc.collect()
torch.cuda.empty_cache()

NameError: name 'trainer' is not defined

In [5]:
""" Initialize the 2D trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer
from utils.config_parser import ConfigParser
import data.threeDPW as module_data
from utils.io import seed_everything

# For fair comparisons
seed_everything(100)

%aimport -ConfigParser # Due to an issue of pickle and auto_reload
config = ConfigParser.wo_args(config=ospj(PROJECT_ROOT,'cfgs/project/gradient-config-hppw.json'))

datamodule = config.init_obj('train_loader', module_data)
train_loader = datamodule.get_loader()

datamodule = config.init_obj('validation_loader', module_data)
val_loader = datamodule.get_loader()

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()


KeyboardInterrupt: 

In [2]:
""" Initialize the 3D trainer and model
"""
import matplotlib.pyplot as plt

from utils.config_parser import ConfigParser
import data.threeDPW as module_data
from utils.io import seed_everything
# For fair comparisons
# seed_everything(599)

%aimport -ConfigParser # Due to an issue of pickle and auto_reload
config = ConfigParser.wo_args(config=ospj(PROJECT_ROOT,'cfgs/project/gradient-config-hppw3d.json'))

datamodule = config.init_obj('train_loader', module_data)
train_loader = datamodule.get_loader()

datamodule = config.init_obj('validation_loader', module_data)
val_loader = datamodule.get_loader()

In [3]:

from trainers.hppw3d_trainer import HPPW3DTrainer

trainer = HPPW3DTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
stats = trainer.train()

plt.plot(stats['loss2d']['train'], label='train2d')
plt.plot(stats['loss2d']['val'], label='val2d')
plt.plot(stats['loss3d']['train'], label='train3d')
plt.plot(stats['loss3d']['val'], label='val3d')
plt.title('Loss Curve')
plt.xlabel('epoch')
plt.ylabel('mpjpe')
plt.legend()
plt.show()

wandb: Currently logged in as: asim-98-12-26 (team-17). Use `wandb login --relogin` to force relogin


==> Start Training Epoch 1/70, lr=0.000085 


Train epoch: 1 loss2d: 2.353164 vim2d: 455.87933: :   8% 448/5760 [00:10<01:10, 75.13it/s]  

KeyboardInterrupt: 

In [3]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

ModuleNotFoundError: No module named 'prettytable'

In [7]:
from torchsummary import summary
from tqdm import tqdm
for i, (history, future) in enumerate(tqdm(val_loader)):
    output = trainer.model(history, future)





  0%|          | 0/203 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [4]:
print(output.shape)

NameError: name 'output' is not defined

In [4]:
!pip install torchinfo

In [5]:
import torchinfo
torchinfo.summary(trainer.model)

Layer (type:depth-idx)                                                      Param #
HumanPosePredictorModel                                                     --
├─PoseEncoder: 1-1                                                          --
│    └─PoseMultiInputSequential: 2-1                                        --
│    │    └─PoseEncoderBlock: 3-1                                           658,432
│    │    └─PoseEncoderBlock: 3-2                                           658,432
│    │    └─PoseEncoderBlock: 3-3                                           658,432
│    │    └─PoseEncoderBlock: 3-4                                           658,432
│    │    └─PoseEncoderBlock: 3-5                                           658,432
│    │    └─PoseEncoderBlock: 3-6                                           658,432
├─VisionTransformer: 1-2                                                    768
│    └─Conv2d: 2-2                                                          (590,592)
│    └─En

In [8]:
model = trainer.model

In [5]:
""" Perform test
"""


datamodule = config.init_obj('test_loader', module_data)
test_loader = datamodule.get_loader()

checkpoint_dir = '0727_002416' 
path = ospj(PROJECT_ROOT, f'saved/models/gradient-HPPW3D/{checkpoint_dir}/per_epoch_model.pth')

trainer.load_model(path=path)




Loading checkpoint: /notebooks/saved/models/gradient-HPPW3D/0727_002416/per_epoch_model.pth ...
Checkpoint loaded.


In [8]:
# checkpoint_dir = '0724_002135' 
# path = ospj(PROJECT_ROOT, f'saved/models/gradient-HPPW3D/{checkpoint_dir}/best_model.pth')
# trainer.load_model(path)
result = trainer.evaluate(loader=test_loader, history_window=15, future_window=15)

print(result)

++> Evaluate at epoch 1 ...


Eval epoch: 1 loss2d: 0.003856 vim2d: 9.89666: : 100% 22624/22624 [09:53<00:00, 38.10it/s]  

++> Evaluate epoch 1 Finished.
{'loss2d': 0.053645996388892316, 'vim2d': 28.453605109596523}


Loading checkpoint: /notebooks/saved/models/gradient-HPPW3D/0722_144744/last_model.pth ...
Checkpoint loaded.


In [34]:
import imageio

for i in range(frames_per_second * video_duration_seconds):
    img = createFrame(i)
    writer.append_data(img)
writer.close()

In [18]:
!pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=5ebdca24bfc77d02e88e3b286d8a5daa4898a1b49e60671e8faef69c9e38b55b
  Stored in directory: /root/.cache/pip/wheels/5a/d9/18/59ee81e4098fc14408ea3d13ae1c472d8380acb97160d2619a
Successfully built ffmpeg


In [8]:
from models.hppw.transforms import cvt_absolute_pose, cvt_relative_pose
from utils.viz import annotate_pose_2d
from utils.viz import annotate_root_2d
import cv2
import numpy as np
from tqdm import tqdm
import time
from subprocess import Popen, PIPE

checkpoint_dir = '0727_002416' 



folder_path = ospj(PROJECT_ROOT, 'data', "outputs", checkpoint_dir)
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

name = ospj(folder_path, checkpoint_dir + ".mp4")

fps, duration = 15, 100
# p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'mjpeg', '-r', str(fps), '-i', '-', '-vcodec', 'mpeg4', '-qscale', '5', '-r', '24', name], stdin=PIPE)

out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc('M','J','P','G'), fps, (224,224))

def _generate_key_padding_mask(poses: torch.Tensor) -> torch.Tensor:
    mask = torch.where(poses==0.0, 1.0, 0.0)

    return torch.sum(mask, dim=-1).bool()

for i, (history, future) in enumerate(tqdm(test_loader)):
            
    img_seq = history[0].float().cuda()
    history_pose2d_seq = history[1].float().cuda()
    history_root_seq = history[2].float().cuda()
    history_mask = history[3].float().cuda()
    
    # mask = torch.zeros_like(history_pose2d_seq)
    # torch.where(x > 0, x, y)
    
    
    history_pose2d_seq = cvt_relative_pose(history_root_seq, history_pose2d_seq)
        

    history_pose2d_seq /= img_seq.shape[3]
    history_root_seq /= img_seq.shape[3]

    
    history_pose2d_mask = _generate_key_padding_mask(history_pose2d_seq)
    history_root_mask = _generate_key_padding_mask(history_root_seq)


    history_pose_mask =  torch.cat([history_root_mask.unsqueeze(-1), history_pose2d_mask], dim=-1)

    with torch.no_grad():
        out_2d, attentions = trainer.model(img_seq, history_pose2d_seq, history_root_seq, history_mask, history_pose_mask, history_root_seq.shape[1])

    pred_root_joints = out_2d[..., 0, :]  * img_seq.shape[3]
    pred_relative_poses = out_2d[..., 1:, :]  * img_seq.shape[3]
    
    pred_abs_poses = cvt_absolute_pose(pred_root_joints, pred_relative_poses)     
    
    
    for sample in range(history[0].shape[0]):
        img = img_seq[sample, -1, ...]
            
        abs_pose = history_pose2d_seq[sample, -1, ...]
        root_joint =history_root_seq[sample, -1, ...]
        
        
        abs_pose = abs_pose.unsqueeze(0).to(torch.uint8)
        root_joint = root_joint.unsqueeze(0).to(torch.uint8)
        # abs_pose = cvt_absolute_pose(root_joint=np.expand_dims(root_joint, 0), norm_pose=np.expand_dims(norm_pose, 0))
        annoted_img = annotate_pose_2d(img=img.cpu().numpy(), pose=abs_pose.cpu().numpy(), color=(255, 0, 0), radius=2, thickness=2, text=False)
        annoted_img = annotate_root_2d(img=annoted_img, root=root_joint.cpu().numpy(), color=(0, 0, 255), thickness=3)
        # cv2.imshow("History Image", img)
        # cv2.imshow("History Mask", mask*255)
        
        
        
        for j in range(future[0].shape[1]):
            
            gt_abs_pose = future[0][sample, j, ...]
            gt_root_joint = future[1][sample, j, ...]
            
            gt_abs_pose = gt_abs_pose.unsqueeze(0).to(torch.uint8)
            gt_root_joint = gt_root_joint.unsqueeze(0).to(torch.uint8)
            
            pred_abs = pred_abs_poses[sample, j, ...]
            pred_root = pred_root_joints[sample, j, ...]
            
            pred_abs = pred_abs.unsqueeze(0).to(torch.uint8)
            pred_root = pred_root.unsqueeze(0).to(torch.uint8)

            # abs_pose = cvt_absolute_pose(root_joint=np.expand_dims(root_joint, 0), norm_pose=np.expand_dims(norm_pose, 0))
            annotated_img = annotate_pose_2d(img=img.cpu().numpy(), pose=pred_abs.cpu().numpy(), color=(225, 225, 0), radius=2, thickness=2, text=False)
            annotated_img = annotate_root_2d(img=annotated_img, root=pred_root.cpu().numpy(), color=(255, 0, 255), thickness=3)
            
            annotated_img = annotate_pose_2d(img=annotated_img, pose=gt_abs_pose.cpu().numpy(), color=(0, 255, 0), radius=2, thickness=2, text=False)
            annotated_img = annotate_root_2d(img=annotated_img,root=gt_root_joint.cpu().numpy(), color=(0, 255, 255), thickness=3)
            # cv2.imwrite(ospj(folder_path, str(i)+".jpg"), annoted_img)

            annotated_img = annotated_img.astype(np.uint8)
            out.write(annotated_img)

    

    
    
out.release()



OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 707/707 [25:52<00:00,  2.20s/it]
wandb: Network error (ReadTimeout), entering retry loop.


In [ ]:
from models.temporal.encoder import TemporalEncoder, LocalTemporalEncoderBlock

model = TemporalEncoder(
    num_layers=3,
    num_heads=8,
    hidden_dim=256,
    mlp_dim=512
)

In [ ]:
local_feat = torch.randn(size=(32, 15, 197, 256))
global_feat = torch.randn(size=(32, 15, 256))

local_out, global_out = model(local_feat, global_feat)

In [ ]:
local_out

tensor([[[ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         ...,
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336]],

        [[-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         ...,
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401]],

        [[-1.1115, -3.7773, -9.5381,  ..., -0.7923,  2.3916, -3.7397],
         [-1.1115, -3.7773, -9.5381,  ..., -0

In [ ]:
from torchsummary import summary

summary(model, input_data=[local_feat, global_feat])

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

In [ ]:

""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
# stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()


KeyError: 'wandb'

In [ ]:
trainer2 = HPPW3DTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_loss2d,█████████▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_vim2d,▇▇▇▇█▇▆▇▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss2d,█████████▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vim2d,█████████▇▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,70
eval_loss2d,0.01851
eval_vim2d,22.99053
loss2d,0.00901
vim2d,12.72046


Adjusting learning rate of group 0 to 8.5000e-05.
